# **Swiss Train Station Analysis**

This notebook demonstrates a comprehensive analysis of Swiss train station data, which contains:

- **Bahnhof**: Station name
- **Kanton**: Swiss canton/province
- **DTV**: Average daily traffic
- **DWV**: Average workday traffic (Mon-Fri)
- **DNWV**: Average non-workday traffic (Sat, Sun, Holidays)
- **EVU**: Train operator
- **lon, lat**: Coordinates

We will walk through:
1. **Data Loading & Cleaning**
2. **Descriptive Statistics & Visualizations** (including how to handle skewed data)
3. **Chi-Squared Test** (Kanton vs. EVU)
4. **One-Way ANOVA** (DTV by Kanton)
5. **Correlation Analysis** (DTV, DWV, DNWV)

We also demonstrate how to handle **outliers** and **log-scale** transformations, given the highly skewed nature of traffic data.

## **Topic**: *Differences in Daily Traffic (DTV) Across Cantons*
We'll see if certain cantons have significantly different average daily traffic, and whether traffic is related to train operator.


In [ ]:
# Standard library imports
import pandas as pd
import numpy as np

from scipy.stats import chi2_contingency, f_oneway, pearsonr, levene, shapiro

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: for Jupyter inline plotting
%matplotlib inline


## 1. Data Loading & Cleaning

Replace `'stations.csv'` below with your actual CSV file if needed.
We assume your CSV has columns:
```
Code,UIC,Bahnhof,Kanton,ISB_GI,Jahr,DTV,DWV,DNWV,EVU,lon,lat
```
and ~3,471 rows. Here we load the data into a pandas DataFrame.


In [ ]:
# Step 1: Load CSV
csv_file = 'stations.csv'  # <-- Change if needed
df = pd.read_csv(csv_file)

print("DataFrame Shape:", df.shape)
df.head(10)

### 1.1 Check for Missing or Invalid Data
We check whether the columns we need are present, and how many missing values exist. If necessary, we can drop or impute them.

In [ ]:
# Checking for missing values
df.isna().sum()

If needed, we can **drop** rows with critical missing columns (e.g., `DTV`, `DWV`, `DNWV`, `Kanton`, `EVU`) or **impute** them. Below we demonstrate dropping them for clarity.

In [ ]:
# We define a list of required columns for analysis
required_cols = ['Kanton','EVU','DTV','DWV','DNWV']
df.dropna(subset=required_cols, inplace=True)

# Convert numeric columns if they're strings
for col in ['DTV','DWV','DNWV','lon','lat']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows that still have NaN in these columns after conversion
df.dropna(subset=['DTV','DWV','DNWV','lon','lat'], inplace=True)

print("Final shape after cleaning:", df.shape)

## 2. Descriptive Statistics & Visualizations
Let's get a sense of the numeric columns, especially how skewed `DTV` might be.

In [ ]:
# Basic descriptive stats for DTV, DWV, DNWV
df[['DTV','DWV','DNWV']].describe()

### 2.1 Distribution of DTV
Traffic data can be **highly skewed**, with many small/medium stations and a few extremely large ones. Let's visualize it.

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=df, x='DTV', kde=True, color='blue')
plt.title('Distribution of DTV (Linear Scale)')
plt.xlabel('DTV')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

Because the data are often **heavily skewed**, you might see a single spike near 0 and a very long tail. Let's also plot it on a **log scale** for better visibility.

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=df, x='DTV', kde=True, color='blue')
plt.xscale('log')
plt.title('Distribution of DTV (Log Scale)')
plt.xlabel('DTV (log scale)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

### 2.2 Descriptive Insights by Canton
Let's look at how many station entries there are per `Kanton`.

In [ ]:
kanton_counts = df['Kanton'].value_counts()
kanton_counts

Visualize counts per canton with a bar plot.

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=kanton_counts.index, y=kanton_counts.values, color='green')
plt.title('Number of Station Rows per Kanton')
plt.xlabel('Kanton')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Chi-Squared Test (Kanton vs. EVU)

We test the **association** between two categorical variables:
- `Kanton` (canton)
- `EVU` (train operator)

**Null Hypothesis (H0)**: There is *no* association between Kanton and EVU.

**Alternative (H1)**: There *is* some association (the distribution of EVU depends on Kanton or vice-versa).

In [ ]:
contingency_table = pd.crosstab(df['Kanton'], df['EVU'])
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print("--- Chi-Squared Test: Kanton vs EVU ---")
print("Chi2 Statistic:", chi2_stat)
print("p-value:", p_value)
print("Degrees of Freedom:", dof)
print("Expected Frequency Table:")
print(expected)

if p_value < 0.05:
    print("\nConclusion: p-value < 0.05 => There is a statistically significant association between Kanton and EVU.")
else:
    print("\nConclusion: p-value >= 0.05 => No strong evidence of association between Kanton and EVU.")

## 4. One-Way ANOVA (DTV by Kanton)

We want to see if the **mean DTV** differs significantly **across cantons**.

**Null Hypothesis (H0)**: All cantons have the *same* mean DTV.

**Alternative (H1)**: At least one canton has a different mean DTV.

### 4.1 Check ANOVA Assumptions
- **Normality** in each group (somewhat robust if sample sizes are large)
- **Homogeneity of variances** across groups


In [ ]:
# Group the data by Kanton
grouped = df.groupby('Kanton')['DTV']
groups_list = [grouped.get_group(k) for k in grouped.groups]

# Levene's Test for equal variances
levene_stat, levene_p = levene(*groups_list)
print("--- Levene's Test for Homogeneity of Variances (DTV by Kanton) ---")
print("Levene Statistic:", levene_stat)
print("p-value:", levene_p)
if levene_p < 0.05:
    print("\nConclusion: p-value < 0.05 => Variances may not be equal.")
else:
    print("\nConclusion: p-value >= 0.05 => Variances are likely equal.")


### 4.2 Perform ANOVA
We use `scipy.stats.f_oneway` for a one-way ANOVA.

In [ ]:
f_stat, p_val = f_oneway(*groups_list)
print("\n--- One-Way ANOVA: DTV by Kanton ---")
print("F-statistic:", f_stat)
print("p-value:", p_val)

if p_val < 0.05:
    print("\nConclusion: p-value < 0.05 => At least one canton has a different mean DTV.")
else:
    print("\nConclusion: p-value >= 0.05 => No significant difference in mean DTV among cantons.")

#### 4.3 (Optional) Post-Hoc Test
If ANOVA is significant, you might want to know *which* cantons differ. You can use tools like **Tukey's HSD** from `statsmodels.sandbox.stats.multicomp`. 
This is optional, but recommended if you get a significant result and want to see pairwise differences.

In [ ]:
# Example (commented out by default):
# !pip install statsmodels  # If not installed
try:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    # Prepare data for Tukey
    dtv_data = df[['Kanton','DTV']].dropna()
    tukey = pairwise_tukeyhsd(endog=dtv_data['DTV'], groups=dtv_data['Kanton'], alpha=0.05)
    print(tukey)
except ImportError:
    print("Install statsmodels to run Tukey's HSD post-hoc test.")

## 5. Correlation Analyses
### 5.1 Correlation between DTV and DWV
We hypothesize DTV (average daily traffic) and DWV (average workday traffic) might be strongly correlated.

In [ ]:
corr_coeff, corr_pval = pearsonr(df['DTV'], df['DWV'])
print("--- Pearson Correlation: DTV vs DWV ---")
print("Correlation Coefficient =", corr_coeff)
print("p-value =", corr_pval)

if corr_pval < 0.05:
    print("Conclusion: p-value < 0.05 => Significant correlation.")
else:
    print("Conclusion: p-value >= 0.05 => Not significant.")

### 5.2 Correlation between DTV and DNWV

In [ ]:
corr_coeff_dnwv, corr_pval_dnwv = pearsonr(df['DTV'], df['DNWV'])
print("\n--- Pearson Correlation: DTV vs DNWV ---")
print("Correlation Coefficient =", corr_coeff_dnwv)
print("p-value =", corr_pval_dnwv)

if corr_pval_dnwv < 0.05:
    print("Conclusion: p-value < 0.05 => Significant correlation.")
else:
    print("Conclusion: p-value >= 0.05 => Not significant.")

### 5.3 Correlation between DWV and DNWV

In [ ]:
corr_coeff_wv_nwv, corr_pval_wv_nwv = pearsonr(df['DWV'], df['DNWV'])
print("\n--- Pearson Correlation: DWV vs DNWV ---")
print("Correlation Coefficient =", corr_coeff_wv_nwv)
print("p-value =", corr_pval_wv_nwv)

if corr_pval_wv_nwv < 0.05:
    print("Conclusion: p-value < 0.05 => Significant correlation.")
else:
    print("Conclusion: p-value >= 0.05 => Not significant.")

### 5.4 Correlation Matrix & Heatmap
A quick overview of correlations among `DTV`, `DWV`, and `DNWV`.

In [ ]:
num_cols = ['DTV','DWV','DNWV']
corr_matrix = df[num_cols].corr(method='pearson')
print("\nCorrelation Matrix:")
print(corr_matrix)

plt.figure(figsize=(5,4))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap: DTV, DWV, DNWV')
plt.tight_layout()
plt.show()

## 6. Additional Insights / Handling Outliers

Given the extreme skew in `DTV`, you might:
1. Use **log-scale** in your plots.
2. **Cap** or **clip** outliers above a certain percentile (e.g., 99th).
3. Investigate large stations individually.

Below is an example of **clipping** at the 99th percentile, then re-plotting the distribution to see the core data more clearly.

In [ ]:
import numpy as np
pct_99 = np.percentile(df['DTV'], 99)
df_clipped = df[df['DTV'] <= pct_99]

plt.figure(figsize=(8,5))
sns.histplot(data=df_clipped, x='DTV', kde=True, color='red')
plt.title('Distribution of DTV (Clipped at 99th Percentile)')
plt.xlabel('DTV (clipped)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

print(f"Clipped at 99th percentile of DTV: {pct_99:.2f}")
print("Shape before clipping:", df.shape, "Shape after clipping:", df_clipped.shape)

## 7. Summary of Findings & Next Steps

1. **Data Cleaning**:
   - We removed rows missing critical columns.
   - Confirmed heavy skew in DTV.
2. **Descriptive Statistics**:
   - Many stations have lower traffic, few have extremely high traffic.
   - Distribution is highly skewed.
3. **Chi-Squared**:
   - We tested whether there's an association between `Kanton` and `EVU`.
   - If p-value < 0.05, there's a significant association.
4. **ANOVA** (DTV by Kanton):
   - We tested if at least one canton differs in mean DTV.
   - If p-value < 0.05, investigate further (e.g., Tukey’s HSD).
5. **Correlation**:
   - We checked how `DTV` relates to `DWV` and `DNWV`.

### Possible Next Steps
- **Filter by Year** (e.g., compare 2022 vs. 2023).
- **Investigate EVU** differences in traffic.
- **Geospatial Analysis**: Plot stations on a map (with `lon` & `lat`) colored by traffic volume. Libraries like `folium` or `geopandas` are handy.
- **Regression Models**: Predict DTV based on location, operator, or other features.

## End of Notebook
We hope this provides a **great in-depth** starting point for your Swiss train station dataset analysis!